<a href="https://colab.research.google.com/github/bnriiitb/whisper-fine-tuning/blob/main/model_evaluation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 11 04:32:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:34:00.0 Off |                    0 |
| N/A   41C    P0   291W / 350W |  28737MiB / 32768MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:36:00.0 Off |                    0 |
| N/A   

In [2]:
import numpy as np
import pandas as pd
import os
from IPython import display
import datasets
from datasets.info import DatasetInfosDict
from datasets import load_dataset,concatenate_datasets, DatasetDict, Dataset, Audio

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
fleurs_dataset = load_dataset("google/fleurs", "te_in", split="test",num_proc=32)

fleurs_dataset = fleurs_dataset.rename_column("transcription", "text")

fleurs_dataset = fleurs_dataset.cast_column("audio",Audio(sampling_rate=16000))

fleurs_dataset = fleurs_dataset.remove_columns(['id', 'num_samples', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_dataset

Found cached dataset fleurs (/raid/cs20mds14030/.cache/huggingface/datasets/google___fleurs/te_in/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


Dataset({
    features: ['path', 'audio', 'text'],
    num_rows: 472
})

In [8]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import soundfile as sf
import torch
from evaluate import load

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# def map_to_pred(batch):
    
#     # print(batch['audio'][0]['array'])
    
#     input_features = processor(batch["audio"]["array"], return_tensors="pt",sampling_rate=16000).input_features

#     with torch.no_grad():
#         logits = model(input_features.to("cuda")).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = processor.batch_decode(predicted_ids, normalize = True)
#     batch['text'] = processor.tokenizer._normalize(batch['text'])
#     batch["transcription"] = transcription
#     return batch

# result = fleurs_dataset.map(map_to_pred)

# wer = load("wer")
# print(wer.compute(predictions=ds["text"], references=ds["transcription"]))

Dataset({
    features: ['audio', 'text'],
    num_rows: 472
})

In [10]:
import time
from tqdm import tqdm
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="bnriiitb/whisper-small-te",language='Telugu')

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe, 
#     inputs=gr.Audio(source="microphone", type="filepath"), 
#     outputs="text",
#     title="Whisper Small Telugu",
#     description="Realtime demo for Telugu speech recognition using a fine-tuned Whisper small model.",
# )

# iface.launch()

In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# sentence  = 'Hello World!'
# tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
# model     = BertModel.from_pretrained('bert-large-uncased')

# inputs    = tokenizer(sentence, return_tensors="pt").to(device)
# pipe     = pipe.to(device)

In [21]:
fleurs_dataset[0]

{'audio': {'path': '7038975664927295261.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00052094, 0.00167996,
         0.00159758]),
  'sampling_rate': 16000},
 'text': 'రెండవ సెట్\u200cలో del potroకు ఆధిక్యం లభించినా కూడా ఈ సెట్\u200cలో కూడా 6-6కి చేరుకున్న తర్వాత టై బ్రేక్ అనివార్యం అయ్యింది'}

In [13]:
pipe(fleurs_dataset[0]['audio']['array'])

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': 'రెండవ సెట్లో డెల్పోట్ర అకు అదిక్యంచనా కోడా  సెట్లో కోడా   నుంచి  చేరుకున్న తర్వాత   వారేం అయింది'}

In [13]:
# !pip install git+https://github.com/openai/whisper.git -q

In [24]:
import whisper
whisper_small_model = whisper.load_model("small",device='cuda')
# whisper_small_model = whisper.load_model("small")

In [16]:
# !conda install pytorch torchvision torchaudio -c pytorch --yes

In [28]:
fleurs_dataset[1]['text']

"హాజరైన వారి సంఖ్య ఎంత ఎక్కువంటే st. peter's స్క్వేర్లో జరిగిన అంత్యక్రియలను చూడడానికి అందరికీ సాధ్యపడలేదు"

In [23]:
display.Audio(fleurs_dataset[1]['audio']['array'],rate=16000)

In [22]:
pipe(fleurs_dataset[1]['audio']['array'])['text']

'హాదరైన వారి  ఎంత ఎక్కువ అంటే సైంట్ పీటర్ స్క్వేడ్    చూడానికి అందరికి సాడ్యపడలేదు'

In [29]:
# whisper_small_model.transcribe(fleurs_dataset[0]['audio']['array'],language="Telugu")
whisper_small_model.transcribe(fleurs_dataset[1]['audio']['array'].astype(np.float32),language="Telugu")['text']

' എകിന്ോയുുെ്െ。 മവുക്ുെ്ു എകാന്്ുു്്ു്്്്് എ്്േ്ൈ്്േ്്്േ്്്്്്േേൔ്്്േ്്്്്്േൃ്ൗ്ൃ്േ'

In [31]:
finetuned_texts = []

for item in tqdm(fleurs_dataset['audio']):
    finetuned_texts.append(pipe(item['array'],language='Telugu')['text'])

finetuned_texts

100%|███████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:20<00:00,  4.05s/it]


['రెండవ సెట్లో డెల్పోట్ర అకు అదిక్యంచనా కోడా  సెట్లో కోడా   నుంచి  చేరుకున్న తర్వాత   వారేం అయింది',
 'హాదరైన వారి  ఎంత ఎక్కువ అంటే సైంట్ పీటర్ స్క్వేడ్    చూడానికి అందరికి సాడ్యపడలేదు',
 'కేవలం కొన్ని స్వలపం బేదాలు తప్ప కాలపనిక బృందాలు కూడా సంప్రదాలు బృందాలు అంటే నైపున్యం కలిగిముంటాయి',
 'ఈ రోస్ మిద్ వారి పర్యటనలో మిగిల సంగీత కచేరి లను రద్దు చేసింది',
 'ఈ ప్రముాధం ఎత్తుగా ఉన్న పర్వత బుబాగంలో జరిగిందని ఇంకా శత్రులు రాజేసిన అగ్ని ఫలితంగా జరిగిందని నమ్ముతారు']

In [ ]:
whisper_langs = []
whisper_texts = []

for item in tqdm(fleurs_dataset['path']):
    pred = whisper_small_model.transcribe(item,language='Telugu')
    whisper_texts.append(pred['text'])
    whisper_langs.append(pred['language'])
whisper_texts

In [38]:
import evaluate
metric = evaluate.load("wer")

finetuned_wer = 100 * metric.compute(predictions=finetuned_texts, references=fleurs_dataset['text'])
whisper_wer = 100 * metric.compute(predictions=whisper_texts, references=fleurs_dataset['text'])

whisper_wer,finetuned_wer

(100.0, 65.15151515151516)